In [ ]:
import os
import sys
import rrdtool
import pickle
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from auserverdata.preparation.rrd import (
    parse_rrds_for_all_collectd_servers,
    parse_rrds_for_all_snmp_servers,
    get_time_series_data_for_collectd_servers,
    get_time_series_data_for_snmp_servers,
    get_feature_names_for_timestamp_set,
    get_feature_names_with_substring_present,
    map_feature_names_to_timestamps,
    map_timestamp_sets_to_feature_names,
    print_timestamp_set_to_feature_names_map_info
)
from auserverdata.local import get_local_config
from auserverdata.analysis import (
    plot_time_series_data_on_single_plot,
    plot_time_series_data_in_vertical_stack
)
from joblib import Parallel,delayed
import pyarrow.parquet as pq
import pyarrow.compute as pc

In [ ]:
local_config = get_local_config()

In [ ]:
# results = parse_rrds_for_all_snmp_servers(snmp_path='/home/bshook/Projects/server-log-api/rrd/original/snmp',
#                                           dst=f'/home/bshook/Projects/server-log-api/rrd/parsed/snmp_parsed_data.parquet')

In [ ]:
# results = parse_rrds_for_all_collectd_servers(collectd_path='/home/bshook/Projects/server-log-api/rrd/original/collectd',
#                                               dst=f'/home/bshook/Projects/server-log-api/rrd/parsed/collectd/parsed_data.parquet',
#                                               n_jobs=-1)

In [ ]:
collectd_ts_df = get_time_series_data_for_collectd_servers(rrd_dir=local_config.rrd_dir, collectd_servers=['gpu18'])

In [ ]:
collectd_ts_df

In [ ]:
collectd_ts_df[('gpu18', 'sensors-coretemp-isa-0001', 'temperature-temp10', 'value')]

In [ ]:
plt.plot(collectd_ts_df[('gpu18', 'sensors-coretemp-isa-0001', 'temperature-temp10', 'value')].dropna())
# plt.yscale('log')

In [ ]:
timestamps_per_feature = map_feature_names_to_timestamps(collectd_ts_df)
features_per_timestamp_set = map_timestamp_sets_to_feature_names(timestamps_per_feature)

In [ ]:
print_timestamp_set_to_feature_names_map_info(features_per_timestamp_set)

In [ ]:
feature_names = get_feature_names_for_timestamp_set(
                    features_per_timestamp_set,
                    timestamp_set = 0
                    )

In [ ]:
len(feature_names)

In [ ]:
feature_names

In [ ]:
cpu_user_feature_names = get_feature_names_with_substring_present(
    feature_names,
    substring = 'memory-used'
    )

In [ ]:
cpu_user_feature_names

In [ ]:
plot_time_series_data_in_vertical_stack(
    collectd_ts_df[cpu_user_feature_names]
    )